In [3]:
from PyQt5 import QtCore
QtCore.QStandardPaths.writableLocation(QtCore.QStandardPaths.AppConfigLocation)

'/var/root/Library/Preferences'

In [4]:
from pathlib import Path

In [27]:
f = Path(QtCore.QStandardPaths.writableLocation(QtCore.QStandardPaths.AppConfigLocation))
f = f / Path("Resolve Mission Control Client")

In [29]:
if not f.exists():
    f.mkdir()

In [31]:
f

PosixPath('/Users/jonnyhyman/Library/Preferences/Resolve Mission Control Client')

In [33]:
!open "/Users/jonnyhyman/Library/Preferences/Resolve Mission Control Client"

In [5]:
import pickle

In [3]:
config = {}

In [48]:
config['auth'] = {}
config['auth']['server_ip'] = ""
config['auth']['client_username'] = ""
config['auth']['server_password'] = ""

In [6]:
with open("testfile.rmc", 'wb') as file:
    pickle.dump(config, file)

In [9]:
with open("testfile.rmc", 'rb') as file:
    configout = pickle.load(file)

In [ ]:
configout

In [49]:
config

{'auth': {'server_ip': '', 'client_username': '', 'server_password': ''}}

In [8]:
class Config:
    """ Configuration file handler class.
        Saves config details to pickled bytes.

        If peak security is required, we could encrypt the bytes of the config
        file (because it holds the server password hash)
    """

    default = {
                'auth' : {},
                'dbses': [],
    }

    def __init__(self, parent):

        saveto = Path(QtCore.QStandardPaths.writableLocation(QtCore.QStandardPaths.AppConfigLocation))
        saveto = saveto.parent / Path("Resolve Mission Control")

        if not saveto.exists():
            saveto.mkdir()

        self.saveto = saveto
        if parent.context == 'client':
            self.fileto = self.saveto / Path("clientconfig.rmc")
        elif parent.context == 'server':
            self.fileto = self.saveto / Path("serverconfig.rmc")
        else:
            raise(ValueError(f"{parent.context} is not a valid context"))
        print(">>>",self.fileto)
        self.config = self.load()

    def __getitem__(self, key):
        """ makes config act like a dict itself to prevent config.config[key] madness """
        return self.config[key]

    def __setitem__(self, key, new_value):
        """ makes config act like a dict itself to prevent config.config[key]=value madness """
        self.config[key] = new_value

    def load(self):

        if not self.fileto.exists():
            return self.default

        try:
            with open(self.fileto, 'rb') as file:
                config = pickle.load(file)
        except EOFError:
            config = self.default

        return config

    def save(self):

        with open(self.fileto, 'wb') as file:
            pickle.dump(self.config, file)